# CNN Implementation in TreNet

### Authors: Nathan Ng

### CNN Implementation

- CNN of *k* filters of window size *w* is applied to raw time series data
- ReLU activation function is applied on CNN outputs
- Results are pooled in max pooling layer 
- Finally, pooled outputs are concatenated to form fully connected layer 

- Note: Dropout layer is skipped because optimal TreNet hyperparameters found for NYSE data had dropout of 0.0

## Setup

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import torch
import torch.nn as nn
import torch.optim as optim

In [2]:
# Use GPU if available for PyTorch
if torch.cuda.is_available():
    dev = "cuda:0"
else:
    dev = "cpu"

device = torch.device(dev)

## Data Pre-Processing

In [3]:
"""
Extracts m sequential data to use to predict n next data 
"""
def extract_data(data, num_input, num_output):
    num_rows = data.shape[0] - num_input - num_output
    input_data = torch.zeros(num_rows, num_input)
    output_data = torch.zeros(num_rows, num_output)
    
    for i in range(num_rows):
        input_data[i] = (data[i:i+num_input])
        output_data[i] = (data[i+num_input:i+num_input+num_output])
    return input_data, output_data

In [4]:
"""
Separates data into train, validation, and test sets
props: (train, valid)
"""
def train_valid_test_split(X, y, props=None):
    if not props: 
        props = (0.5, 0.25)
    elif len(props) != 3: 
        print("Wrong number of parameters")
        return None
    
    train_size = int(X.shape[0] * props[0])
    valid_size = int(X.shape[0] * props[1])
    
    X_train = X[:train_size].to(device)
    y_train = y[:train_size].to(device)
    
    X_valid = X[train_size: (train_size + valid_size)].to(device)
    y_valid = y[train_size: (train_size + valid_size)].to(device)
    
    X_test = X[(train_size + valid_size):].to(device)
    y_test = y[(train_size + valid_size):].to(device)
    
    return X_train, y_train, X_valid, y_valid, X_test, y_test

## TreNet CNN Model

In [5]:
class TreNetCNN(nn.Module):
    def __init__(self, num_data, layers=None, num_filters=None, dropout=None, conv_size=3, pooling_size=3, output_size=2):
        """
        layers (int): Number of cnn stacks to create 
        num_filters (list(int)): Number of CNN filters corresponding to same index stacks
        dropout (list(float)): Probability of dropout corresponing to same index stack
        convsize (int, list(int)): Size of filter sizes 
        """
        super(TreNetCNN, self).__init__()
        self.num_data = num_data
        self.layers = layers
        self.num_filters = num_filters
        self.dropout = dropout
        self.conv_size = conv_size
        self.pooling_size = pooling_size
        self.output_size = output_size
        self.cnn_stack = self.create_cnn_stack()
    
    def create_cnn_stack(self):
        # Initialize default stack settings
        if not self.layers:
            self.layers = 2
        if not self.num_filters:
            self.num_filters = [128] * self.layers
        if not self.dropout:
            self.dropout = [0.0] * self.layers
        if type(self.conv_size) == int:
            self.conv_size = [self.conv_size] * self.layers
        
        # Create cnn stacks 
        cnn_stacks = []
        num_channels = 1
        updated_data = self.num_data
        for i in range(self.layers):
            cnn_stack = nn.Sequential(
                nn.Conv1d(in_channels=num_channels, out_channels=self.num_filters[i], kernel_size=self.conv_size[i]),
                nn.ReLU(),
                nn.MaxPool1d(kernel_size=self.pooling_size, stride=1),
                nn.Dropout(p=self.dropout[i])
            )
            num_channels = self.num_filters[i]
            cnn_stacks.append(cnn_stack)
            
            # Keep track of current size of data 
            updated_data = updated_data - self.conv_size[i] + 1 - self.pooling_size + 1
            new_data_size = self.num_filters[i] * updated_data
            
        # Add fully connected layer at end to output trend duration and slope
        output_layer = nn.Sequential(
            nn.Flatten(), 
            nn.Linear(new_data_size, self.output_size)
        )
        cnn_stacks.append(output_layer)
            
        # Combine cnn stacks 
        return nn.Sequential(*cnn_stacks)
    
    def forward(self, x):
        x = torch.reshape(x, (1, 1, -1))
        output = self.cnn_stack(x)
        return output

## TreNet CNN Training and Loss Function

In [6]:
"""
Create training loop
"""
def train_loop(X, y, model, loss_fn, optimizer):
    for i in range(X.shape[0]):
        # Compute prediction and loss 
        pred = model(X[i])
        loss = loss_fn(pred, y[i])
        
        # Perform backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

In [7]:
"""
Create test loop
"""
def test_loop(X, y, model, loss_fn):
    test_loss = 0
    
    with torch.no_grad():
        for i in range(X.shape[0]):
            pred = model(X[i])
            test_loss += loss_fn(pred, y[i]).item()
            
    test_loss /= X.shape[0]

    return test_loss

In [8]:
"""
RMSE Loss Function
"""
def rmse(output, target):
    loss = torch.mean((output - target)**2)
    return torch.sqrt(loss)

## Test CNN Stack Model on Raw Stock Prices

In [9]:
# Load in NYSE data
df = pd.read_csv("../data/raw/indexProcessed.csv")
NYSE_df = df[df["Index"] == "NYA"].loc[:, "Open"].reset_index(drop=True)

# Create subset of data 
NYSE_sub = NYSE_df[:1000]
sub = torch.tensor(NYSE_sub).to(device)

In [10]:
# Separate train, validation, and test sets
num_outputs = 1
X, y = extract_data(sub, 100, num_outputs)
train_X, train_y, valid_X, valid_y, test_X, test_y = train_valid_test_split(X, y)

In [11]:
# Set hyperparameters for training 
learning_rate = 0.000001
batch_size = 64
epochs = 35

# Create model
model = TreNetCNN(100, layers = 2, num_filters = [32, 32], dropout=[0.1, 0.1], conv_size=[2, 4], output_size=num_outputs).to(device)

# Initialize loss function 
loss_fn = rmse

# Initialize optimizer 
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [12]:
train_score = []
valid_score = []
for i in range(epochs):
    train_loop(train_X, train_y, model, loss_fn, optimizer)
    train_score.append(test_loop(train_X, train_y, model, loss_fn))
    valid_score.append(test_loop(valid_X, valid_y, model, loss_fn))
    
    if i % 5 == 0 or i == epochs-1: 
        print(f"Epoch {i}:\n------------------")
        print(f"Train Loss: {train_score[-1]}")
        print(f"Validation Loss: {valid_score[-1]}")
        print()
    
print("Done!")

/opt/conda/lib/python3.9/site-packages/torch/nn/functional.py:652: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool1d(input, kernel_size, stride, padding, dilation, ceil_mode)


Epoch 0:
------------------
Train Loss: 383.7760111037766
Validation Loss: 450.33145345960344

Epoch 5:
------------------
Train Loss: 33.17122379974161
Validation Loss: 37.70672471182687

Epoch 10:
------------------
Train Loss: 33.53104059096169
Validation Loss: 39.09117957523891

Epoch 15:
------------------
Train Loss: 32.23318223167369
Validation Loss: 37.02065413338797

Epoch 20:
------------------
Train Loss: 32.072252099391875
Validation Loss: 35.54894038609096

Epoch 25:
------------------
Train Loss: 31.61235770244641
Validation Loss: 34.038603918892996

Epoch 30:
------------------
Train Loss: 33.4190853943007
Validation Loss: 32.31206784929548

Epoch 34:
------------------
Train Loss: 32.466567483404965
Validation Loss: 35.42890957423619

Done!
